In [1]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants, analysis
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
import matplotlib.pyplot as plt
import matplotlib as mpl
pd.options.mode.chained_assignment = None
%matplotlib inline
#plt.style.use('ggplot')
#mpl.rcParams['figure.figsize'] = (10,6)
#mpl.rcParams['figure.dpi'] = 150

In [2]:
pudl_engine  = pudl.connect_db()

In [3]:
ferc_plants_all = ferc1_plant_ids = pd.read_sql('''SELECT plant_id_pudl FROM plants_ferc''', pudl_engine)

In [4]:
ferc_plants_simple = analysis.simple_ferc1_plant_ids(pudl_engine)

In [5]:
ferc_plants_complex = ferc_plants_all[~ferc_plants_all.plant_id_pudl.isin(ferc_plants_simple.plant_id_pudl)]

In [6]:
expns_corrs, steam_complex = analysis.ferc_expenses(pudl_engine,pudl_plant_ids=ferc_plants_complex.plant_id_pudl)

In [7]:
expns_corrs, steam_simple = analysis.ferc_expenses(pudl_engine,pudl_plant_ids=ferc_plants_simple.plant_id_pudl)

In [8]:
expns_corrs, steam_all = analysis.ferc_expenses(pudl_engine,pudl_plant_ids=ferc_plants_all.plant_id_pudl)

In [9]:
steam_all.drop_duplicates(subset=['plant_id_pudl']).describe()

,report_year,respondent_id,util_id_pudl,plant_id_pudl,total_capacity_mw,year_constructed,year_installed,peak_demand_mw,water_limited_mw,not_water_limited_mw,...,expns_allowances,expns_engineering,expns_structures,expns_boiler,expns_plants,expns_misc_steam,expns_production_total,expns_per_mwh,expns_total_nonfuel_production,expns_total_nonproduction
count,724.000000,724.000000,724.000000,724.000000,724.000000,718.000000,711.000000,724.000000,724.000000,724.000000,...,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02,7.240000e+02
mean,2005.252762,121.790055,196.750000,572.332873,462.497251,1975.941504,1981.403657,396.835635,256.171271,350.779006,...,1.740824e+05,8.362947e+05,4.869734e+05,2.800857e+06,1.364125e+06,9.597027e+05,4.586774e+07,1.529741e+04,9.517000e+06,3.427432e+06
std,2.730075,75.267215,102.627741,1084.798967,563.104385,21.979563,17.154277,510.875851,453.291959,513.432999,...,7.563042e+05,2.886650e+06,1.287096e+06,7.415145e+06,2.720466e+06,3.560830e+06,7.008498e+07,2.450475e+05,2.183649e+07,9.522171e+06
min,2004.000000,1.000000,7.000000,1.000000,0.000000,1900.000000,1943.000000,-1.000000,0.000000,0.000000,...,-1.404710e+05,-4.577000e+03,-6.792457e+06,-2.756910e+06,-1.491360e+05,-3.069800e+04,-9.883100e+06,-5.500000e+00,-3.950400e+05,-2.558631e+07
25%,2004.000000,56.000000,106.000000,199.750000,72.000000,1963.000000,1969.000000,35.000000,0.000000,0.000000,...,0.000000e+00,1.710000e+02,3.602500e+02,0.000000e+00,4.909925e+04,6.175000e+01,1.382375e+06,2.127500e+01,1.839880e+05,8.200050e+04
50%,2004.000000,127.000000,190.000000,399.500000,254.430000,1973.000000,1978.000000,205.000000,16.000000,121.500000,...,0.000000e+00,4.526850e+04,6.105200e+04,4.575150e+04,3.904965e+05,9.185600e+04,1.259642e+07,5.525000e+01,1.944545e+06,8.364505e+05
75%,2005.000000,177.000000,285.500000,603.250000,623.425000,1995.000000,1999.000000,563.500000,338.500000,516.250000,...,0.000000e+00,4.215360e+05,4.273695e+05,2.570009e+06,1.426323e+06,5.621352e+05,6.468411e+07,1.499250e+02,8.796485e+06,2.743277e+06
max,2016.000000,432.000000,369.000000,10296.000000,4704.000000,2016.000000,2016.000000,2966.000000,3080.000000,3090.000000,...,1.067697e+07,3.573339e+07,1.491494e+07,1.351145e+08,2.612138e+07,5.910080e+07,3.959891e+08,6.232949e+06,2.709273e+08,9.641457e+07


In [10]:
steam_complex.drop_duplicates(subset=['plant_id_pudl']).describe()

,report_year,respondent_id,util_id_pudl,plant_id_pudl,total_capacity_mw,year_constructed,year_installed,peak_demand_mw,water_limited_mw,not_water_limited_mw,...,expns_allowances,expns_engineering,expns_structures,expns_boiler,expns_plants,expns_misc_steam,expns_production_total,expns_per_mwh,expns_total_nonfuel_production,expns_total_nonproduction
count,290.000000,290.000000,290.000000,290.000000,290.000000,289.000000,285.000000,290.000000,290.000000,290.000000,...,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02,2.900000e+02
mean,2004.731034,112.865517,185.368966,449.431034,475.483379,1973.062284,1978.354386,408.975862,288.306897,382.451724,...,2.149431e+05,9.324601e+05,5.660486e+05,3.087830e+06,1.350566e+06,1.048495e+06,5.030704e+07,7.983257e+03,1.020567e+07,3.837271e+06
std,1.946535,77.912731,104.264827,324.406955,555.455036,19.959236,15.779422,505.276049,484.976187,524.008465,...,9.236022e+05,2.620859e+06,1.389085e+06,6.075879e+06,2.529443e+06,3.069267e+06,7.213044e+07,6.951890e+04,1.893832e+07,9.981262e+06
min,2004.000000,1.000000,7.000000,1.000000,10.000000,1906.000000,1943.000000,-1.000000,0.000000,0.000000,...,-1.404710e+05,0.000000e+00,-6.792457e+06,-7.362610e+05,0.000000e+00,-1.154000e+04,-9.883100e+06,-5.500000e+00,-2.706420e+05,-2.558631e+07
25%,2004.000000,44.000000,91.000000,200.250000,86.375000,1961.000000,1968.000000,47.250000,0.000000,14.250000,...,0.000000e+00,3.692000e+03,1.651500e+03,0.000000e+00,7.300025e+04,3.571000e+03,2.126652e+06,1.955000e+01,3.581175e+05,1.515112e+05
50%,2004.000000,108.000000,171.000000,408.500000,265.200000,1971.000000,1974.000000,234.000000,51.500000,184.500000,...,0.000000e+00,1.034545e+05,1.194155e+05,6.070740e+05,3.959300e+05,1.682160e+05,2.165321e+07,4.525000e+01,3.298194e+06,1.166216e+06
75%,2004.000000,161.000000,277.000000,596.750000,642.000000,1985.000000,1988.000000,596.500000,408.750000,547.000000,...,0.000000e+00,5.618992e+05,5.939100e+05,3.234353e+06,1.386288e+06,7.516560e+05,7.116891e+07,1.412750e+02,1.139771e+07,3.176057e+06
max,2014.000000,432.000000,369.000000,1284.000000,3279.600000,2014.000000,2014.000000,2966.000000,3080.000000,3090.000000,...,1.067697e+07,2.350062e+07,1.491494e+07,5.345957e+07,2.265142e+07,3.843895e+07,3.959891e+08,1.065781e+06,1.868855e+08,9.641457e+07


In [11]:
steam_simple.drop_duplicates(subset=['plant_id_pudl']).describe()

,report_year,respondent_id,util_id_pudl,plant_id_pudl,total_capacity_mw,year_constructed,year_installed,peak_demand_mw,water_limited_mw,not_water_limited_mw,...,expns_allowances,expns_engineering,expns_structures,expns_boiler,expns_plants,expns_misc_steam,expns_production_total,expns_per_mwh,expns_total_nonfuel_production,expns_total_nonproduction
count,434.000000,434.000000,434.000000,434.000000,434.000000,429.000000,426.000000,434.000000,434.000000,434.000000,...,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02,4.340000e+02
mean,2005.601382,127.753456,204.354839,654.456221,453.819885,1977.881119,1983.443662,388.723502,234.698157,329.615207,...,1.467792e+05,7.720368e+05,4.341352e+05,2.609100e+06,1.373186e+06,9.003713e+05,4.290140e+07,2.018475e+04,9.056831e+06,3.153576e+06
std,3.100339,72.929949,100.925708,1370.336151,568.630978,23.062437,17.742945,515.004955,430.061131,505.745705,...,6.195740e+05,3.052652e+06,1.212965e+06,8.190154e+06,2.843793e+06,3.856842e+06,6.860813e+07,3.114161e+05,2.358550e+07,9.204203e+06
min,2004.000000,1.000000,7.000000,2.000000,0.000000,1900.000000,1949.000000,-1.000000,0.000000,0.000000,...,-5.830000e+04,-4.577000e+03,-1.530000e+02,-2.756910e+06,-1.491360e+05,-3.069800e+04,0.000000e+00,0.000000e+00,-3.950400e+05,-3.228966e+06
25%,2004.000000,61.000000,121.000000,199.750000,69.250000,1964.000000,1970.000000,30.250000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,5.050000e+01,0.000000e+00,3.920050e+04,0.000000e+00,8.973315e+05,2.400000e+01,1.223442e+05,7.172000e+04
50%,2004.000000,132.000000,203.000000,390.000000,245.100000,1974.000000,1980.000000,189.500000,0.000000,72.000000,...,0.000000e+00,2.294750e+04,3.061350e+04,0.000000e+00,3.656170e+05,3.408550e+04,9.449214e+06,6.325000e+01,1.285966e+06,6.564440e+05
75%,2005.000000,181.000000,292.000000,616.750000,616.965000,2000.000000,2001.000000,538.000000,312.000000,478.250000,...,0.000000e+00,3.708585e+05,3.096190e+05,1.858628e+06,1.475815e+06,4.665460e+05,5.697681e+07,1.535000e+02,7.117584e+06,2.320704e+06
max,2016.000000,307.000000,369.000000,10296.000000,4704.000000,2016.000000,2016.000000,2617.000000,2438.000000,2538.000000,...,6.072514e+06,3.573339e+07,1.297804e+07,1.351145e+08,2.612138e+07,5.910080e+07,3.796632e+08,6.232949e+06,2.709273e+08,9.118243e+07


In [12]:
# Convert the generation_eia923 table into a dataframe
g9 = analysis.simple_select('generation_eia923', pudl_engine)
# Get yearly net generation by plant_id, year and generator_id
g9_summed = analysis.yearly_sum_eia(g9, 'net_generation_mwh')
g9_summed.reset_index(inplace=True)

In [13]:
# Convert the boiler_generator_assn_eia860 table into a dataframe
bga8 = analysis.simple_select('boiler_generator_assn_eia860', pudl_engine)
bga8.drop(['id','operator_id'],axis=1, inplace=True)
bga8.drop_duplicates(['plant_id','boiler_id','generator_id'],inplace=True)

In [14]:
# Convert the generators_eia860 table into a dataframe
g8 = analysis.simple_select('generators_eia860', pudl_engine)
g8 = g8[['plant_id','plant_name', 'operator_name', 'state', 'generator_id','year']]
g8.rename(columns={'year': 'report_date'},inplace=True)

In [15]:
# All cenerators from the Boiler Generator Association table (860)
gens8 = bga8.drop_duplicates(subset=['plant_id','generator_id'])
# All cenerators from the generation table (923)
gens9 = g9_summed.drop_duplicates(subset=['plant_id','generator_id','report_date'])
# All generators
gens = gens9.merge(gens8, on=['plant_id','generator_id'], how="left")
gens = gens.merge(g8, on=['plant_id','generator_id','report_date'],how='left')
# Bring in pudl IDs for all EIA plants
pudl_ids = g9 = analysis.simple_select('plants_eia', pudl_engine)
# Get pudl_ids into gen list
gens = gens.merge(pudl_ids, how='left',on=['plant_id','plant_name'])

In [16]:
gens.rename(columns={'plant_name':'plant_name_eia',
                     'plant_id':'plant_id_eia',
                     'operator_name':'operator_name_eia'}, inplace=True)

In [17]:
steam_complex.rename(columns={'report_year': 'report_date',
                              'plant_name':'plant_name_ferc',
                              'net_generation_mwh':'net_generation_mwh_ferc'},
                     inplace=True)

In [18]:
steam_complex[steam_complex['plant_id_pudl'] == 234]

,report_date,respondent_id,util_id_pudl,respondent_name,plant_id_pudl,plant_name_ferc,total_capacity_mw,year_constructed,year_installed,peak_demand_mw,...,expns_allowances,expns_engineering,expns_structures,expns_boiler,expns_plants,expns_misc_steam,expns_production_total,expns_per_mwh,expns_total_nonfuel_production,expns_total_nonproduction
9534,2014,288,339,"UNS Electric, Inc.",234,Gila River,619.00,2003.0,2003.0,0.0,...,0.0,0.0,3253.0,8434.0,0.0,0.0,92090.0,224.6,23332.0,48244.0
9544,2014,176,330,Tucson Electric Power Company,234,Gila River,619.00,2003.0,2003.0,0.0,...,0.0,0.0,9759.0,0.0,25302.0,0.0,271830.0,459.9,69997.0,170419.0
10125,2015,176,330,Tucson Electric Power Company,234,Gila River,464.25,2003.0,2003.0,351.0,...,0.0,0.0,137448.0,0.0,2340613.0,0.0,45615062.0,27.8,2679406.0,2993902.0
10128,2015,288,339,"UNS Electric, Inc.",234,Gila River,155.00,2003.0,2003.0,117.0,...,0.0,0.0,26377.0,0.0,620612.0,0.0,14136889.0,26.5,672242.0,401425.0
10990,2016,176,330,Tucson Electric Power Company,234,Gila River,464.25,2003.0,2003.0,379.0,...,0.0,0.0,98861.0,0.0,1653547.0,0.0,45007037.0,25.1,2000953.0,2549052.0
10993,2016,288,339,"UNS Electric, Inc.",234,Gila River,155.00,2003.0,2003.0,126.0,...,0.0,0.0,9427.0,0.0,171889.0,0.0,13487154.0,23.3,214472.0,277719.0


In [19]:
gens_ferc_eia_merge = gens.merge(steam_complex, how='left',on=['plant_id_pudl','report_date'])

In [22]:
gens_ferc_eia_merge.describe()

,plant_id_eia,report_date,net_generation_mwh,respondent_id,util_id_pudl,total_capacity_mw,year_constructed,year_installed,peak_demand_mw,water_limited_mw,...,expns_allowances,expns_engineering,expns_structures,expns_boiler,expns_plants,expns_misc_steam,expns_production_total,expns_per_mwh,expns_total_nonfuel_production,expns_total_nonproduction
count,30571.000000,30571.000000,3.057100e+04,5973.000000,5973.000000,5973.000000,5894.000000,5799.000000,5973.000000,5973.000000,...,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03,5.973000e+03
mean,20925.642472,2012.750581,8.044898e+05,118.221664,186.436632,583.266864,1977.645741,1982.479910,505.728947,331.320777,...,2.758487e+05,8.711988e+05,8.889074e+05,4.940650e+06,2.198829e+06,8.757996e+05,8.509272e+07,3.018568e+03,1.411119e+07,5.039429e+06
std,23810.951386,2.157422,1.300407e+06,87.673913,109.739828,627.783922,18.602436,15.851988,681.052242,521.308521,...,1.857385e+06,2.698123e+06,1.949298e+06,8.765077e+06,3.247280e+06,1.837787e+06,1.180661e+08,1.085577e+05,2.297057e+07,1.699070e+07
min,3.000000,2009.000000,-2.874970e+05,1.000000,7.000000,0.000000,1906.000000,1948.000000,-8.000000,0.000000,...,-5.541160e+06,-1.850663e+06,-6.529540e+05,-1.638790e+05,-1.952901e+07,-3.942872e+06,-3.125400e+04,-1.860000e+02,-4.622760e+05,-3.784949e+06
25%,2123.000000,2011.000000,4.024000e+04,56.000000,96.000000,132.000000,1967.000000,1970.000000,95.000000,0.000000,...,0.000000e+00,4.067000e+03,2.668300e+04,0.000000e+00,2.064490e+05,1.191600e+04,8.488816e+06,3.130000e+01,1.190208e+06,3.400730e+05
50%,6165.000000,2013.000000,2.416030e+05,108.000000,169.000000,366.000000,1975.000000,1980.000000,298.000000,78.000000,...,0.000000e+00,2.348690e+05,2.704050e+05,1.517520e+06,9.274200e+05,2.936950e+05,4.283871e+07,4.110000e+01,7.107813e+06,1.759724e+06
75%,50884.000000,2015.000000,9.372040e+05,170.000000,287.000000,782.480000,1993.000000,1999.000000,639.000000,485.000000,...,6.140000e+02,8.714390e+05,9.145770e+05,5.688304e+06,2.914807e+06,8.655120e+05,1.114312e+08,6.810000e+01,1.584577e+07,3.936253e+06
max,60768.000000,2016.000000,1.030283e+07,454.000000,369.000000,3499.000000,2015.000000,2015.000000,11052.000000,3066.000000,...,3.611004e+07,6.379013e+07,2.622739e+07,7.087108e+07,3.649845e+07,2.971917e+07,9.902384e+08,5.876767e+06,3.552832e+08,3.032670e+08


In [23]:
mcoe_annotations = pd.read_csv('mcoe_field_annotations_complex_ferc.csv')
mcoe_notes = pd.read_csv('mcoe_notes_complex_ferc.csv')

In [24]:
# to export all the data
xlsx_writer = pd.ExcelWriter('EIA_mcoe_inputs_complex_plants.xlsx')
gens_ferc_eia_merge.to_excel(xlsx_writer, sheet_name='FERC Expenses by EIA Generator', index=False, na_rep='NA')
mcoe_annotations.to_excel(xlsx_writer, sheet_name='MCOE Annotations', index=False)
mcoe_notes.to_excel(xlsx_writer, sheet_name='MCOE Notes', index=False)

xlsx_writer.save()